## Training a variational quantum circuit

We explain this part through an example

### Importing modules

In [10]:
import pennylane as qml
from pennylane import numpy as np 
from qiskit import*
import math as math
import numpy as np
from numpy import pi as pi
from numpy import sqrt as sqrt
import math as math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pennylane.templates.embeddings import AmplitudeEmbedding
from sklearn.model_selection import train_test_split

In [4]:
dev = qml.device('default.qubit', wires=10, shots=20000)

Our data is the catalogue data from SDSS Dr16. u,g,r,i,z are the fluxes in respective bands, redshift is the redshift of the object, and class is 0 if the object is star and 1 if its a galaxy

In [57]:
data_o = pd.read_csv("400_data.csv")
data_o[:300]

,Unnamed: 0,u,g,r,i,z,class,redshift
0,0,18.69254,17.13867,16.55555,16.34662,16.17639,0.0,0.000115
1,26,19.13669,17.43355,17.09256,16.91690,16.58885,1.0,0.019020
2,34,18.72582,17.54478,17.06506,16.88649,16.79680,0.0,-0.000015
3,39,19.16617,17.57986,16.80475,16.35483,16.01940,1.0,0.061022
4,49,18.95918,17.09173,16.25019,15.83413,15.55686,1.0,0.064456
...,...,...,...,...,...,...,...,...
295,2604,18.25147,16.98479,16.52890,16.63696,16.33264,0.0,0.000080
296,2606,18.81737,17.59944,17.09213,16.87947,16.80449,0.0,0.000575
297,2610,18.78798,17.27448,16.46987,16.00137,15.67138,1.0,0.075507
298,2615,18.95853,17.36799,16.61208,16.27441,15.99602,1.0,0.027828


In [18]:
N=6

In [58]:
data = data_o[[ 'u', 'g', 'r', 'i', 'z', 'redshift','class']][:300]
data

,u,g,r,i,z,redshift,class
0,18.69254,17.13867,16.55555,16.34662,16.17639,0.000115,0.0
1,19.13669,17.43355,17.09256,16.91690,16.58885,0.019020,1.0
2,18.72582,17.54478,17.06506,16.88649,16.79680,-0.000015,0.0
3,19.16617,17.57986,16.80475,16.35483,16.01940,0.061022,1.0
4,18.95918,17.09173,16.25019,15.83413,15.55686,0.064456,1.0
...,...,...,...,...,...,...,...
295,18.25147,16.98479,16.52890,16.63696,16.33264,0.000080,0.0
296,18.81737,17.59944,17.09213,16.87947,16.80449,0.000575,0.0
297,18.78798,17.27448,16.46987,16.00137,15.67138,0.075507,1.0
298,18.95853,17.36799,16.61208,16.27441,15.99602,0.027828,1.0


Performing data preprocessing

In [59]:
xscaler = MinMaxScaler((0,np.pi))
X_train, X_test, y_train, y_test = train_test_split(data.drop(['class'],1), data['class'], test_size=0.25, random_state=42)
Xs_train = xscaler.fit_transform(X_train)
y_train,y_test=y_train.values,y_test.values
Xs_test = xscaler.transform(X_test)

C:\Users\Dheeru\AppData\Local\Temp/ipykernel_16404/1030090167.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train, X_test, y_train, y_test = train_test_split(data.drop(['class'],1), data['class'], test_size=0.25, random_state=42)


Creating our circuit where we encode the data directly using the RY gate, where data values are scaled between 0 to pi

In [60]:
@qml.qnode(dev)
def circuit(data, oom):

    for i in range(N):
        qml.RY(data[i], wires=i)
    
    for i in range(N):
        qml.RX(oom[i], wires=i)
    
    for i in range(N-1):
        qml.CNOT(wires=[i,i+1])
    qml.CNOT(wires=[5,0])
    
    for i in range(N):
        qml.RY(oom[N+i], wires=i)
    
    ex=[]
    '''for i in range(N):
        ex.append(qml.expval(qml.PauliX(i)))
    print(ex)
    e=np.mean(ex)np.bincount(x).argmax()'''
    return qml.probs(wires=range(N))
    #return [qml.expval(qml.PauliX(i)) for i in range(N)]

In [61]:
drawer=qml.draw(circuit)
oom=np.random.uniform(0,np.pi,N*2)
print(drawer(Xs_train[0],oom))

 0: ──RY(1.71)─────RX(0.69)────╭C────────────────────────────────────────────────────╭X──RY(2.32)──╭┤ Probs 
 1: ──RY(0.619)────RX(2.22)────╰X──╭C───RY(0.887)────────────────────────────────────│─────────────├┤ Probs 
 2: ──RY(0.259)────RX(0.815)───────╰X──╭C───────────RY(2.62)─────────────────────────│─────────────├┤ Probs 
 3: ──RY(0.407)────RX(0.222)───────────╰X──────────╭C──────────RY(0.54)──────────────│─────────────├┤ Probs 
 4: ──RY(0.7)──────RX(0.0497)──────────────────────╰X─────────╭C─────────RY(0.0377)──│─────────────├┤ Probs 
 5: ──RY(0.00247)──RX(2.32)───────────────────────────────────╰X─────────────────────╰C──RY(2.3)───╰┤ Probs 



In [62]:
def prediction(data,param):
    probs=circuit(data,param)
    return 1 if sum(probs[:len(probs)//2])<0.5 else 0
def cost(param):
    cost = 0
    for k in range(len(Xs_train)):
        cost += abs(prediction(Xs_train[i],param) - y_train[k])
    return cost

In [51]:
y_train.values

array([1., 1., 0., ..., 1., 1., 0.])

We use the Gradient descent optimiser to optimise the cost function

In [66]:
opt = qml.GradientDescentOptimizer(stepsize = 0.05)
steps = 15
param = np.random.uniform(0,np.pi,N*2)

In [67]:
cost(param) # initial cost fucntion

79.0

In [68]:
param

array([0.79004712, 0.75489367, 2.73584442, 1.11784902, 2.14994019,
       0.9910167 , 0.92081763, 1.69120228, 1.5815369 , 1.86374752,
       1.63516858, 1.61725341])

In [69]:
for i in range(steps):
    param = opt.step(cost, param)
    if (i+1)%5 ==0:
        print("cost funtion on step {:5d}: {: .7f}".format( i+1, cost(param)))
print('optimized rotation angle: {}'.format(param))

cost funtion on step     5:  79.0000000
cost funtion on step    10:  79.0000000
cost funtion on step    15:  79.0000000
optimized rotation angle: [0.79004712 0.75489367 2.73584442 1.11784902 2.14994019 0.9910167
 0.92081763 1.69120228 1.5815369  1.86374752 1.63516858 1.61725341]


Currently the optimization has some problems.. We are trying to solve this